# Explore here

Loading the Data into Pandas DataFrames 

check tables

In [2]:
import pandas as pd 

movies = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv')
credits = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv')

print(movies.head(5))
print(credits.head(5))

      budget                                             genres  \
0  237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1  300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2  245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3  250000000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4  260000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   

                                       homepage      id  \
0                   http://www.avatarmovie.com/   19995   
1  http://disney.go.com/disneypictures/pirates/     285   
2   http://www.sonypictures.com/movies/spectre/  206647   
3            http://www.thedarkknightrises.com/   49026   
4          http://movies.disney.com/john-carter   49529   

                                            keywords original_language  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...                en   
1  [{"id": 270, "name": "ocean"}, {"id": 726, "na...                en   
2  [{"id": 470, "nam

In [3]:
movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [4]:
credits.columns

Index(['movie_id', 'title', 'cast', 'crew'], dtype='object')

Creating a Database and Storing DataFrames as Tables

In [5]:
import sqlite3
sql_database = sqlite3.connect('movies&credits.sqlite')

movies.to_sql('movies', sql_database, index=False)
credits.to_sql('credits', sql_database, index=False)

ValueError: Table 'movies' already exists.

In [36]:
join_tables = """
SELECT movies.*, credits.cast, credits.crew
FROM movies
JOIN credits ON movies.title = credits.title;
"""

joined_df = pd.read_sql_query(join_tables, sql_database)

print(joined_df.head())

      budget                                             genres  \
0  237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1  300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2  245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3  250000000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4  260000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   

                                       homepage      id  \
0                   http://www.avatarmovie.com/   19995   
1  http://disney.go.com/disneypictures/pirates/     285   
2   http://www.sonypictures.com/movies/spectre/  206647   
3            http://www.thedarkknightrises.com/   49026   
4          http://movies.disney.com/john-carter   49529   

                                            keywords original_language  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...                en   
1  [{"id": 270, "name": "ocean"}, {"id": 726, "na...                en   
2  [{"id": 470, "nam

In [47]:
gs = joined_df['genres']

In [50]:
import json
json.loads(gs[0])

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [8]:
sql_database.commit()

In [ ]:
sql_database.close()

CLEANing the JoiNED TABLE TO LEAVE ONLY:

movie_id, 
title, 
overview, 
genres, 
keywords, 
cast, 
crew

In [14]:
import json

def extract_names(json_str):
    if json_str and isinstance(json_str, str):
        try:
            return ', '.join(item['name'] for item in json.loads(json_str))
        except json.JSONDecodeError:
            return None  # Return None if JSON is malformed
    return None  # Return None if json_str is empty or not a string

def extract_cast_names(json_str):
    if json_str and isinstance(json_str, str):
        try:
            return ', '.join(item['name'] for item in json.loads(json_str)[:3])
        except json.JSONDecodeError:
            return None
    return None

def extract_director_name(json_str):
    if json_str and isinstance(json_str, str):
        try:
            crew = json.loads(json_str)
            for item in crew:
                if item.get('job') == 'Director':
                    return item.get('name')
        except json.JSONDecodeError:
            return None
    return None



In [20]:
joined_df['genres'] = joined_df['genres'].apply(extract_names)
joined_df['keywords'] = joined_df['keywords'].apply(extract_names)
joined_df['cast'] = joined_df['cast'].apply(extract_cast_names)
joined_df['crew'] = joined_df['crew'].apply(extract_director_name)

print(joined_df.head())

      budget genres                                      homepage      id  \
0  237000000   None                   http://www.avatarmovie.com/   19995   
1  300000000   None  http://disney.go.com/disneypictures/pirates/     285   
2  245000000   None   http://www.sonypictures.com/movies/spectre/  206647   
3  250000000   None            http://www.thedarkknightrises.com/   49026   
4  260000000   None          http://movies.disney.com/john-carter   49529   

  keywords original_language                            original_title  \
0     None                en                                    Avatar   
1     None                en  Pirates of the Caribbean: At World's End   
2     None                en                                   Spectre   
3     None                en                     The Dark Knight Rises   
4     None                en                               John Carter   

                                            overview  popularity  \
0  In the 22nd century, 

In [35]:
print(joined_df['genres'])

0       None
1       None
2       None
3       None
4       None
        ... 
4804    None
4805    None
4806    None
4807    None
4808    None
Name: genres, Length: 4809, dtype: object


remove the spaces between the words and concatenate names to split similar named keywords. Apply this function to the columns genres, cast, crew and keywords.

In [23]:
def remove_spaces(s):
    if isinstance(s, str):
        return ','.join(word.replace(" ", "") for word in s.split(','))
    return s

print(joined_df.head())

      budget genres                                      homepage      id  \
0  237000000   None                   http://www.avatarmovie.com/   19995   
1  300000000   None  http://disney.go.com/disneypictures/pirates/     285   
2  245000000   None   http://www.sonypictures.com/movies/spectre/  206647   
3  250000000   None            http://www.thedarkknightrises.com/   49026   
4  260000000   None          http://movies.disney.com/john-carter   49529   

  keywords original_language                            original_title  \
0     None                en                                    Avatar   
1     None                en  Pirates of the Caribbean: At World's End   
2     None                en                                   Spectre   
3     None                en                     The Dark Knight Rises   
4     None                en                               John Carter   

                                            overview  popularity  \
0  In the 22nd century, 

In [22]:
joined_df['tags'] = joined_df['genres'] + ',' + joined_df['cast'] + ',' + joined_df['crew'] + ',' + joined_df['keywords']
joined_df['tags'] = joined_df['tags'].str.replace(',', ' ')

print(joined_df[['tags']])

     tags
0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
...   ...
4804  NaN
4805  NaN
4806  NaN
4807  NaN
4808  NaN

[4809 rows x 1 columns]


BUILD A KNN TO CLuSTER SIMILAR CATEGORIES

VECTORIZE TO GET FROM TEXT TO NUMERICAL

In [31]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors


joined_df['tags'] = joined_df['tags'].fillna('')

# Proceed with CountVectorizer as before
count_vectorizer = CountVectorizer(stop_words='english',max_features=500)
count_matrix = count_vectorizer.fit_transform(joined_df['tags'])





ValueError: empty vocabulary; perhaps the documents only contain stop words

In [ ]:
from sklearn

In [ ]:
knn_model = NearestNeighbors(n_neighbors=5, metric='cosine')
knn_model.fit(count_matrix)
y_pred = 

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity()